1\. Write a function that converts number representation (bin<->dec<->hex)

In [13]:
def repr_num(x, mode:str) -> int:
    """
    Converts x value's representation into another
    x : int or str
    mode : str = "bin", "dec", "hex"
    """
    if type(x) == str: # aka bin or hex
        if x[:2] == "0x":
            x = int(x,16)
        elif x[:2] == "0b":
            x = int (x,2)
        else:
            try:
                x = int(x)
            except:
                print(f"Error: {x} is not an integer value.")
                return
    elif type(x) != int:
        print(f"Error: {x} is not an integer value.")
        return
    conversions = {"bin":bin, "dec":int, "hex": hex}
    if not mode in conversions.keys():
        print(f"Error: {mode} is not a valid representation. Please pass one of the following as argument:\n" \
        "bin, dec, hex")
        return
    conversions = {"bin":bin, "dec":int, "hex": hex}
    return conversions[mode](x)

for x in (0, 0b10101010101, -16, "a", hex(10), 1.1):
    print (repr_num(x, "bin"), repr_num(x, "dec"), repr_num(x, "hex"))
    

0b0 0 0x0
0b10101010101 1365 0x555
-0b10000 -16 -0x10
Error: a is not an integer value.
Error: a is not an integer value.
Error: a is not an integer value.
None None None
0b1010 10 0xa
Error: 1.1 is not an integer value.
Error: 1.1 is not an integer value.
Error: 1.1 is not an integer value.
None None None


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [1]:
bias = 127

def convert(word:str) -> float:
    print("\n")
    if type(word) != str:
        print(f"Error: {word} is not a string.")
        return
    if len(word) > 32:
        print(f"Error: {word} is not a 32-bit word.")
        return
    for bit in word:
        if not bit in "01":
            print(f"Error: {word} is not a binary word.")
            return
    print(f"Converting word: {word}")
    # filling word of zeros if less than 32-bit
    word = "0"*(32-len(word)) + word
    # extracting components
    sign = word[0]
    exponent = word[1:9]
    mantissa = word[9:]
    print(f"sign: {sign}, exponent: {exponent}, mantissa: {mantissa}")
    # checking special cases
    if exponent == "1"*8:
        print("Warning: special case.")
        if mantissa != "0"*23:
            print("Converted number is NaN")
            return
        else: # f > 0
            if sign == "0":
                print("Converted number is +inf")
                return
            else:
                print("Converted number is -inf")
                return
    # parsing components
    print("Converting word's components into decimal...")
    sign = 1 if sign == "0" else -1
    exponent = int(exponent, 2)
    mantissa = int(mantissa,2)
    # First addend is checking for subnormal number
    mantissa = (exponent != 0) + mantissa / 2**23
    print(f"Decimal representation: sign: {sign}, exponent: {exponent}, mantissa: {mantissa:.15f}")
    # printing result
    number = sign * mantissa * 2**(exponent-bias)
    print(f"Converted number is {number}")

for number in ("0"*32, "10101010", "11000000101100000000000000000000",
               "0"+"11111110"+"1"*23, "1"*9+"0"*23, "1"*32, "1"*40, "13"):
    convert(number)




Converting word: 00000000000000000000000000000000
sign: 0, exponent: 00000000, mantissa: 00000000000000000000000
Converting word's components into decimal...
Decimal representation: sign: 1, exponent: 0, mantissa: 0.000000000000000
Converted number is 0.0


Converting word: 10101010
sign: 0, exponent: 00000000, mantissa: 00000000000000010101010
Converting word's components into decimal...
Decimal representation: sign: 1, exponent: 0, mantissa: 0.000020265579224
Converted number is 1.1911036946760945e-43


Converting word: 11000000101100000000000000000000
sign: 1, exponent: 10000001, mantissa: 01100000000000000000000
Converting word's components into decimal...
Decimal representation: sign: -1, exponent: 129, mantissa: 1.375000000000000
Converted number is -5.5


Converting word: 01111111011111111111111111111111
sign: 0, exponent: 11111110, mantissa: 11111111111111111111111
Converting word's components into decimal...
Decimal representation: sign: 1, exponent: 254, mantissa: 1.9999998

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [36]:
# Python integers don't have upper bound
# We can try using floats
x, y = 1., 1.
while x*2 != float('inf'):
    x *=2
while y/2 != 0.:
    y /= 2
# Forcing scientific formatting because of too few digits for low value numbers
print(f"overflow limit for Python floats: {x}, underflow limit for floats: {y:e}")

# We can also try for numpy aka C-style precision
import numpy as np
print("Numpy version:")
# numpy ints
a = np.int_(1)
# Overflow causes negative value. This check gives runtime warning but 
# I'm leaving it here because it's what we're searching for
while a*2 > 0:
    a *=2
print(f"overflow limit for integers: {a}")
# numpy floats
x, y = np.float_(1), np.float_(1)
# numpy overflow causes inf value
while x*2 != np.inf:
    x *=2
while y/2 != 0:
    y /= 2
print(f"overflow limit for floats: {x}, underflow limit for floats: {y:e}")

overflow limit for Python floats: 8.98846567431158e+307, underflow limit for floats: 4.940656e-324
Numpy version:
overflow limit for integers: 1073741824
overflow limit for floats: 8.98846567431158e+307, underflow limit for floats: 4.940656e-324


<ipython-input-36-6b4186c03faf>:18: RuntimeWarning: overflow encountered in long_scalars
  while a*2 > 0:
<ipython-input-36-6b4186c03faf>:24: RuntimeWarning: overflow encountered in double_scalars
  while x*2 != np.inf:


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [57]:
x = 1
e = 1
while x+e/2 != x:
    e /=2
print(f"Machine precision is: {e}")
# We check result with epsilon from sys.float_info
import sys
epsilon = sys.float_info.epsilon
if epsilon == e:
    print("Machine precision computation is correct")
else:
    print("Error: machine precision computation is incorrect")

Machine precision is: 2.220446049250313e-16
Machine precision computation is correct


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [125]:
# decorator for checking parameters and printing results
def check_and_print(f):
    def wrapper(a, b, c):
        if {type(a), type(b), type(c)} != {int,float}:
            print("Error: can't parse input parameters.")
            return
        x1, x2 = f(a, b, c)
        global results_storage
        results_storage += [x1, x2]
        print(f"The two solutions are: x1 = {x1:.15e}, x2 = {x2:.15e}")
    return wrapper

@check_and_print
def solve_a(a, b, c):
    x1 = (-b - (b**2-4*a*c)**0.5) / (2*a)
    x2 = (-b + (b**2-4*a*c)**0.5) / (2*a)
    return x1, x2

# for data analysis later
results_storage = []
parameters = (0.001, 1000, 0.001)
print("First method:")
solve_a(*parameters)

@check_and_print
def solve_b(a, b, c):
    x1 = 4*a*c / (2*a*(-b + (b**2-4*a*c)**0.5))
    x2 = 4*a*c / (2*a*(-b - (b**2-4*a*c)**0.5))
    return x1, x2

print("\n")
print("Second method:")
solve_b(*parameters)

# Few data elaborations
rs = results_storage
err_perc_1 = (rs[2]-rs[0])/rs[0]
err_perc_2 = (rs[3]-rs[1])/rs[1]
print("\n")
print(f"Absoulte differences between the two methods are: {rs[2]-rs[0]} for x1, {rs[3]-rs[1]} for x2.")
print(f"Relative differences between the two methods are: {err_perc_1} for x1, {err_perc_2} for x2.")

# We try now to achieve a better precision avoiding computing difference between neighboring numbers:
@check_and_print
def solve_c(a, b, c):
    x1 = (-b - (b**2-4*a*c)**0.5) / (2*a)
    x2 = 4*a*c / (2*a*(-b - (b**2-4*a*c)**0.5))
    return x1, x2
print("\n")
print("Precise alternative method:")
solve_c(*parameters)

# We can try decimal module too
from decimal import Decimal
@check_and_print
def solve_decimal_a(a, b, c):
    a = Decimal(a)
    b = Decimal(b)
    c = Decimal(c)
    x1 = (-b - (b**Decimal(2)-Decimal(4)*a*c)**Decimal(0.5)) / (Decimal(2)*a)
    x2 = (-b + (b**Decimal(2)-Decimal(4)*a*c)**Decimal(0.5)) / (Decimal(2)*a)
    return x1, x2

@check_and_print
def solve_decimal_b(a, b, c):
    a = Decimal(a)
    b = Decimal(b)
    c = Decimal(c)
    x1 = Decimal(4)*a*c / (Decimal(2)*a*(-b + (b**Decimal(2)-Decimal(4)*a*c)**Decimal(0.5)))
    x2 = Decimal(4)*a*c / (Decimal(2)*a*(-b - (b**Decimal(2)-Decimal(4)*a*c)**Decimal(0.5)))
    return x1, x2

print("\n")
print("Trying with decimal module")
print("First method:")
solve_decimal_a(*parameters)
print("Second method:")
solve_decimal_b(*parameters)

# Other few data elaborations
print("\n")
print(f"Absoulte differences between the alternative method and 1st Decimal method are: \
{Decimal(rs[4])-rs[6]} for x1, {Decimal(rs[5])-rs[7]} for x2.")
print(f"Relative differences between the alternative method and 1st Decimal method are: \
{Decimal(rs[4])/rs[6] - Decimal(1)} for x1, {Decimal(rs[5])/rs[7] - Decimal(1)} for x2.")
# We expect very similiar differences with 2nd Decimal method
print(f"Absoulte differences between the alternative method and 2nd Decimal method are: \
{Decimal(rs[4])-rs[8]} for x1, {Decimal(rs[5])-rs[9]} for x2.")
print(f"Relative differences between the alternative method and 2nd Decimal method are: \
{Decimal(rs[4])/rs[8] - Decimal(1)} for x1, {Decimal(rs[5])/rs[9] - Decimal(1)} for x2.")

"""
x1 2nd method < x1 1st method
x2 2nd method < x2 1st method
Differences are "small": relative differences of ~10^(-5) for x1 and x2
In the computation of x2 with 1st method and in the computation of x1 with 2nd method,
the problem is the quantity -b + sqrt(b^2 - 4ac) = -b + (b-e), e small. That is because b >> a,c =>
=> computing difference between two neighboring numbers => difference result has low precision
"""
    

First method:
The two solutions are: x1 = -9.999999999990000e+05, x2 = -9.999894245993346e-07


Second method:
The two solutions are: x1 = -1.000010575512506e+06, x2 = -1.000000000001000e-06


Absoulte differences between the two methods are: -10.575513505726121 for x1, -1.0575401665279555e-11 for x2.
Relative differences between the two methods are: 1.0575513505736698e-05 for x1, 1.0575513505572119e-05 for x2.


Precise alternative method:
The two solutions are: x1 = -9.999999999990000e+05, x2 = -1.000000000001000e-06


Trying with decimal module
First method:
The two solutions are: x1 = -9.999999999990000e+5, x2 = -1.000000000001000e-6
Second method:
The two solutions are: x1 = -9.999999999990000e+5, x2 = -1.000000000001000e-6


Absoulte differences between the alternative method and 1st Decimal method are: -1.320218934144384765625E-11 for x1, 1.520409727605634711596740090E-22 for x2.
Relative differences between the alternative method and 1st Decimal method are: 1.3202189341E-17 for

'\nx1 2nd method < x1 1st method\nx2 2nd method < x2 1st method\nDifferences are "small": relative differences of ~10^(-5) for x1 and x2\nIn the computation of x2 with 1st method and in the computation of x1 with 2nd method,\nthe problem is the quantity -b + sqrt(b^2 - 4ac) = -b + (b-e), e small. That is because b >> a,c =>\n=> computing difference between two neighboring numbers => difference result has low precision\n'

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [144]:
def f(x):
    return x*(x-1)

delta = 1e-2
def derivative(x):
    return (f(x+delta) - f(x)) / delta

print(f"The derivative of f(x) = x(x-1) for x = 1 is: f'(1) = {derivative(1)}")
# The value obtained differs from true value (f'(1) = 1) because
# we're adding/subtracting neighboring values in computation

# accuracy = 1/(f'(1, delta) - 1)
accuracies = [1/(derivative(1)-1)]
for delta in (1e-4, 1e-6, 1e-8, 1e-12, 1e-14):
    print(f"delta = {delta} => f'(1) = {derivative(1)}")
    accuracies.append(1/(derivative(1)-1))

# Accuracy reaches a maximum for delta = 1e-08
print(f"Accuracies: {accuracies}")

The derivative of f(x) = x(x-1) for x = 1 is: f'(1) = 1.010000000000001
delta = 0.0001 => f'(1) = 1.0000999999998899
delta = 1e-06 => f'(1) = 1.0000009999177333
delta = 1e-08 => f'(1) = 1.0000000039225287
delta = 1e-12 => f'(1) = 1.0000889005833413
delta = 1e-14 => f'(1) = 0.9992007221626509
Accuracies: [99.99999999999103, 10000.000011014514, 1000082.273488583, 254937583.55597517, 11248.520115561294, -1251.1293986539047]


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [3]:
import math
import sys
import pickle
import timeit
from numba import jit, cuda, prange
import numpy as np

MAX_ITERATIONS = 10000 # just an upper limit for loop iterations

# For storing variables that take too much time to compute
class ResultsClass(object):
    def __init__(self):
        # Triggers for algorithms
        self.run_1sec_done = False
        self.run_1min_done = False
        self.run_1sec_parallel_cpu_done = False
        self.run_1min_parallel_cpu_done = False
        self.run_1sec_parallel_cuda_done = False
        self.run_1min_parallel_cuda_done = False
        # Initial values of N and integrals' values found with algorithms
        self.N_1sec = 100
        self.I_1sec = 0
        self.N_1min = 100
        self.I_1min = 0
        self.N_1sec_parallel_cpu = 100
        self.I_1sec_parallel_cpu = 0
        self.N_1min_parallel_cpu = 100
        self.I_1min_parallel_cpu = 0
        #!!! GPU part is CUDA = just NVIDIA cards
        self.N_1sec_parallel_cuda = 100
        self.I_1sec_parallel_cuda = 0
        self.N_1min_parallel_cuda = 100
        self.I_1min_parallel_cuda = 0

# Loading previous data
try:
    with open('02ex_data.pkl', 'rb') as fid:
        results = pickle.load(fid)
except:
    results = ResultsClass()

# Integrand function f(x) = sqrt(1-x^2)
def f(x):
    return (1-x**2)**0.5

# Function that Riemann-integrates f(x) given N subdivisions in [-1,1]
def integrate(N):
    h = 2/N
    s = 0
    for i in range(N):
        s += f(-1 + i*h) * h
    return s

def relative_diff(integral):
    return integral/(math.pi/2) - 1

N = 100
I = integrate(N)
print(f"For N = {N} result is: {I}")
print(f"\tIt has a relative difference of {relative_diff(I)}")
# Result is lesser than true value and it is not much accurate

### Part 1 - Finding N_1sec, I_1sec
if not results.run_1sec_done:
    N_1sec = 100
    I_1sec = 0
    # We now find max N that takes less than 1 sec on this PC (within a factor of 2)
    i = 0
    while True:
        i += 1
        if i > MAX_ITERATIONS:
            print("Warning: MAX_ITERATIONS exceeded. Breaking from loop.")
            break
        N_1sec_temp = N_1sec * 2

        start = timeit.default_timer()
        I_1sec_temp = integrate(N_1sec_temp)
        end = timeit.default_timer()

        if end - start < 1:
            N_1sec = N_1sec_temp
            I_1sec = I_1sec_temp
        else:
            break
    results.N_1sec = N_1sec
    results.I_1sec = I_1sec
    results.run_1sec_done = True
    with open('02ex_data.pkl', 'wb') as fid:
        pickle.dump(results, fid)
print(f"Max N for running time < 1 sec is N = {results.N_1sec}, with integral I = {results.I_1sec}")
print(f"\tIt has a relative difference of {relative_diff(results.I_1sec)}")

### Part 2 - Finding N_1min, I_1min
# We now find max N that takes less than 1 minute on this PC (within a factor of 2)
print("--- Pure Python ---")
if not results.run_1min_done:
    i = 0
    N_1min = results.N_1sec
    I_1min = results.I_1sec
    while True:
        i += 1
        if i > MAX_ITERATIONS:
            print("Warning: MAX_ITERATIONS exceeded. Breaking from loop.")
            break
        N_1min_temp = N_1min * 2

        start = timeit.default_timer()
        I_1min_temp = integrate(N_1min)
        end = timeit.default_timer()

        if end - start < 60:
            N_1min = N_1min_temp
            I_1min = I_1min_temp
        else:
            break
    results.N_1min = N_1min
    results.I_1min = I_1min
    results.run_1min_done = True
    with open('02ex_data.pkl', 'wb') as fid:
        pickle.dump(results, fid)
print(f"Max N for running time < 1 min is N = {results.N_1min}, with integral I = {results.I_1min}")
print(f"\tIt has a relative difference of {relative_diff(results.I_1min)}")
print(f"\tThat's a gain of {abs(relative_diff(results.I_1sec)/relative_diff(results.I_1min))*100}%")

### Part 3 - Finding N_1sec_parallel_cpu, I_1sec_parallel_cpu
# We now use JIT compilation + parallelism on CPU power to achieve faster execution times
print("--- On CPU ---")
@jit
def f_cpu(x):
    return (1-x**2)**0.5

@jit
def integrate_cpu(N):
    h = 2/N
    s = 0
    for i in prange(N):
        s += f_cpu(-1 + i*h) * h
    return s

if not results.run_1sec_parallel_cpu_done:
    N_1sec_parallel_cpu = 100
    I_1sec_parallel_cpu = 0
    i = 0
    while True:
        i += 1
        if i > MAX_ITERATIONS:
            print("Warning: MAX_ITERATIONS exceeded. Breaking from loop.")
            break
        N_1sec_parallel_cpu_temp = N_1sec_parallel_cpu * 2

        start = timeit.default_timer()
        I_1sec_parallel_cpu_temp = integrate_cpu(N_1sec_parallel_cpu_temp)
        end = timeit.default_timer()

        if end - start < 1:
            N_1sec_parallel_cpu = N_1sec_parallel_cpu_temp
            I_1sec_parallel_cpu = I_1sec_parallel_cpu_temp
        else:
            break
    results.N_1sec_parallel_cpu = N_1sec_parallel_cpu
    results.I_1sec_parallel_cpu = I_1sec_parallel_cpu
    results.run_1sec_parallel_cpu_done = True
    with open('02ex_data.pkl', 'wb') as fid:
        pickle.dump(results, fid)
print(f"Max N for running time < 1 sec is N = {results.N_1sec_parallel_cpu}, \
with integral I = {results.I_1sec_parallel_cpu}")
print(f"\tIt has a relative difference of {relative_diff(results.I_1sec_parallel_cpu)}")

### Part 4 - Finding N_1min_parallel_cpu, I_1min_parallel_cpu
if not results.run_1min_parallel_cpu_done:
    i = 0
    N_1min_parallel_cpu = results.N_1sec_parallel_cpu
    I_1min_parallel_cpu = results.I_1sec_parallel_cpu
    while True:
        i += 1
        if i > MAX_ITERATIONS:
            print("Warning: MAX_ITERATIONS exceeded. Breaking from loop.")
            break
        N_1min_parallel_cpu_temp = N_1min_parallel_cpu * 2

        start = timeit.default_timer()
        I_1min_parallel_cpu_temp = integrate_cpu(N_1min_parallel_cpu)
        end = timeit.default_timer()

        if end - start < 60:
            N_1min_parallel_cpu = N_1min_parallel_cpu_temp
            I_1min_parallel_cpu = I_1min_parallel_cpu_temp
        else:
            break
    results.N_1min_parallel_cpu = N_1min_parallel_cpu
    results.I_1min_parallel_cpu = I_1min_parallel_cpu
    results.run_1min_parallel_cpu_done = True
    with open('02ex_data.pkl', 'wb') as fid:
        pickle.dump(results, fid)
print(f"Max N for running time < 1 min is N = {results.N_1min_parallel_cpu}, \
with integral I = {results.I_1min_parallel_cpu}")
print(f"\tIt has a relative difference of {relative_diff(results.I_1min_parallel_cpu)}")
print(f"\tThat's a gain of \
{relative_diff(results.I_1sec_parallel_cpu)/relative_diff(results.I_1min_parallel_cpu)*100}%")
print("We can see how for N too much high we lose in accuracy.")

### Part 5 - Finding N_1sec_parallel_cuda, I_1sec_parallel_cuda
# Standard GPUs don't have much accuracy, this code is more for 64bit GPUs, and maybe for DPUs in the future
# However, in many situations (ex. features extracting), code run on GPU is too much useful
print("--- On GPU (CUDA) ---")
# GPU info
try:
    cuda.detect()
    cuda_context = cuda.current_context()
except:
    print("Error: current system doesn't support CUDA. Exiting...")
    exit(1)
print("Free/Total Memory on GPU:", cuda_context.get_memory_info())

# Step 1/2: we calculate first the area under a step h
@cuda.jit()
def integrate_cuda_partial(vec_in, vec_out):
    h = 2/vec_in.shape[0]
    for i in prange(vec_in.shape[0]):
        vec_out[i] = (1-vec_in[i]**2)**0.5 * h
    cuda.syncthreads()

# Step 2/2: we sum all "partial" sums
@cuda.reduce
def integrate_cuda(a, b):
    return a + b

if not results.run_1sec_parallel_cuda_done:
    N_1sec_parallel_cuda = 100
    I_1sec_parallel_cuda = results.I_1sec_parallel_cuda
    i = 0
    while True:
        i += 1
        if i > MAX_ITERATIONS:
            print("Warning: MAX_ITERATIONS exceeded. Breaking from loop.")
            break
        N_1sec_parallel_cuda_temp = N_1sec_parallel_cuda * 2

        cuda.profile_start()
        start = timeit.default_timer()
        
        threadsperblock = 32
        blockspergrid = (N_1sec_parallel_cuda_temp + (threadsperblock - 1)) // threadsperblock
        vec = np.arange(-1, 1, 2/N_1sec_parallel_cuda_temp)
        vec_out = np.zeros(vec.size)
        integrate_cuda_partial[blockspergrid, threadsperblock](vec, vec_out)
        I_1sec_parallel_cuda_temp = integrate_cuda(vec_out)
        
        end = timeit.default_timer()
        cuda.profile_stop()

        if end - start < 1:
            N_1sec_parallel_cuda = N_1sec_parallel_cuda_temp
            I_1sec_parallel_cuda = I_1sec_parallel_cuda_temp
        else:
            break
    results.N_1sec_parallel_cuda = N_1sec_parallel_cuda
    results.I_1sec_parallel_cuda = I_1sec_parallel_cuda
    results.run_1sec_parallel_cuda_done = True
    with open('02ex_data.pkl', 'wb') as fid:
        pickle.dump(results, fid)
print(f"Max N for running time < 1 sec is N = {results.N_1sec_parallel_cuda}, \
with integral I = {results.I_1sec_parallel_cuda}")
print(f"\tIt has a relative difference of {relative_diff(results.I_1sec_parallel_cuda)}")
print("We shouldn't use GPU to parallelize too few operations because they're much slower than on CPU. \
Situation is different in situations like next step for very high N.")

### Part 6 - Finding N_1min_parallel_cuda, I_1min_parallel_cuda
if not results.run_1min_parallel_cuda_done:
    i = 0
    N_1min_parallel_cuda = results.N_1sec_parallel_cuda
    I_1min_parallel_cuda = results.I_1sec_parallel_cuda
    while True:
        i += 1
        if i > MAX_ITERATIONS:
            print("Warning: MAX_ITERATIONS exceeded. Breaking from loop.")
            break
        N_1min_parallel_cuda_temp = N_1min_parallel_cuda * 2

        cuda.profile_start()
        start = timeit.default_timer()
        
        threadsperblock = 32
        blockspergrid = (N_1min_parallel_cuda_temp + (threadsperblock - 1)) // threadsperblock
        vec = np.arange(-1, 1, 2/N_1min_parallel_cuda_temp)
        vec_out = np.zeros(vec.size)
        integrate_cuda_partial[blockspergrid, threadsperblock](vec, vec_out)
        I_1min_parallel_cuda_temp = integrate_cuda(vec_out)
        
        end = timeit.default_timer()
        cuda.profile_stop()

        if end - start < 60:
            N_1min_parallel_cuda = N_1min_parallel_cuda_temp
            I_1min_parallel_cuda = I_1min_parallel_cuda_temp
        else:
            break
    results.N_1min_parallel_cuda = N_1min_parallel_cuda
    results.I_1min_parallel_cuda = I_1min_parallel_cuda
    results.run_1min_parallel_cuda_done = True
    with open('02ex_data.pkl', 'wb') as fid:
        pickle.dump(results, fid)
print(f"Max N for running time < 1 min is N = {results.N_1min_parallel_cuda}, \
with integral I = {results.I_1min_parallel_cpu}")
print(f"\tIt has a relative difference of {relative_diff(results.I_1min_parallel_cuda)}")
print(f"\tThat's a gain of \
{abs(relative_diff(results.I_1sec_parallel_cuda)/relative_diff(results.I_1min_parallel_cuda))*100}%")
print("\n")
print("The winner in these cases seems to be CPU with ~1sec run, this because GPU utilized has not \
much precision and for N very large we have h very small and we lose precision due to bits representation.")

For N = 100 result is: 1.5691342555492505
	It has a relative difference of -0.0010581074180618932
Max N for running time < 1 sec is N = 1638400, with integral I = 1.5707963260017854
	It has a relative difference of -5.049102247411952e-10
--- Pure Python ---
Max N for running time < 1 min is N = 209715200, with integral I = 1.57079632679301
	It has a relative difference of -1.2009282457370318e-12
	That's a gain of 42043.329943607285%
--- On CPU ---
Max N for running time < 1 sec is N = 419430400, with integral I = 1.570796326795702
	It has a relative difference of 5.127009927718973e-13
Max N for running time < 1 min is N = 53687091200, with integral I = 1.5707963270944043
	It has a relative difference of 1.906725888289884e-10
	That's a gain of 0.26889077025735025%
We can see how for N too much high we lose in accuracy.
--- On GPU (CUDA) ---
Found 1 CUDA devices
id 0    b'GeForce GTX 1660 SUPER'                              [SUPPORTED]
                      compute capability: 7.5
      